In [1]:
!pip install -q evaluate transformers[torch]

In [2]:
# Load a dataset from Hugging Face
from datasets import load_dataset

hf_dataset_identifier = "jacquelinegrimm/arabidopsis-kmeans"
ds = load_dataset(hf_dataset_identifier)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/jacquelinegrimm--arabidopsis-kmeans-dc8875be3ba5f03a/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
# Shuffle and split the dataset into training and test sets

ds = ds.shuffle(seed=1) # The 'seed' parameter is set to 1 for reproducibility
ds = ds["train"].train_test_split(test_size=0.2)

train_ds = ds["train"]
test_ds = ds["test"]

In [4]:
import json
from huggingface_hub import hf_hub_download

In [5]:
# To configure the model, extract the number of unique labels

repo_id = f"datasets/{hf_dataset_identifier}"
filename = "id2label.json" # Define the filename containing label information

# Download the 'id2label.json' file
id2label = json.load(open(hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"), "r"))

# Convert the keys in 'id2label' from strings to integers and store the result in 'id2label'
id2label = {int(k): v for k, v in id2label.items()}

# Create a 'label2id' dictionary by reversing the key-value pairs in 'id2label'
label2id = {v: k for k, v in id2label.items()}

# Calculate the number of unique labels in the dataset
num_labels = len(id2label)

id2label.json:   0%|          | 0.00/36.0 [00:00<?, ?B/s]

In [6]:
from torchvision.transforms import ColorJitter
from transformers import SegformerImageProcessor

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2024-02-05 14:01:20.005490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-05 14:01:20.005587: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-05 14:01:20.297354: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
# Transform data to match the model's expected input shape and perform image augmentation
# Apply color jitter, which introduces random variations in brightness, contrast, saturation, and hue

processor = SegformerImageProcessor()
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

# Function to apply transformations to a batch of training examples
def train_transforms(example_batch):

    # Apply color jitter to each image in the batch
    images = [jitter(x) for x in example_batch['pixel_values']]
    # Extract labels from the batch
    labels = [x for x in example_batch['label']]
    # Use the processor to transform images and labels to match the expected input shape
    inputs = processor(images, labels)
    # Return the transformed inputs
    return inputs

# Function to apply transformations to a batch of validation examples
def val_transforms(example_batch):

    # Extract images from the batch without applying color jitter
    images = [x for x in example_batch['pixel_values']]
    # Extract labels from the batch
    labels = [x for x in example_batch['label']]
    # Use the processor to transform images and labels to match the expected input shape
    inputs = processor(images, labels)
    # Return the transformed inputs
    return inputs

# Set the transformation functions for the training and testing datasets
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [8]:
# Load the semantic segmentation model
from transformers import SegformerForSemanticSegmentation

pretrained_model_name = "jacquelinegrimm/segformer-b0-finetuned-arabidopsis-roots-v02" 

model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id
)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

In [9]:
# Specify training arguments and name the fine-tuned model
from transformers import TrainingArguments

# Define training hyperparameters
epochs = 50
lr = 0.00006
batch_size = 2

hub_model_id = "segformer-b0-finetuned-arabidopsis-roots-v03" # Name the fine-tuned model

training_args = TrainingArguments(
    "segformer-b0-finetuned-arabidopsis-roots-v03-outputs",  # Name the output directory
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id=hub_model_id,
    hub_strategy="end",
)

In [10]:
# Define mean intersection over union (IoU) as the evaluation metric
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")

# Function to calculate evaluation metrics for model predictions
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred

        # Convert 'logits' into a tensor and resize it to match the shape of "labels"
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        # Convert the predicted labels to a NumPy array
        pred_labels = logits_tensor.detach().cpu().numpy()

        # Calculate metrics using the 'mean_iou' evaluation metric
        metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),  # Number of unique labels in the dataset
            ignore_index=0,
            reduce_labels=processor.do_reduce_labels,
        )

        # Extract per-category accuracy and IoU scores
        per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
        per_category_iou = metrics.pop("per_category_iou").tolist()

        # Update the metrics dictionary with accuracy and IoU scores for each category
        metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
        metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

        return metrics

In [11]:
# Log in to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [12]:
# Train the model
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Background,Accuracy Seedling,Iou Background,Iou Seedling
20,0.047700,0.037365,0.432796,0.865593,0.865593,nan,0.865593,0.000000,0.865593
40,0.035500,0.033744,0.443057,0.886114,0.886114,nan,0.886114,0.000000,0.886114
60,0.039100,0.032790,0.441585,0.883170,0.883170,nan,0.883170,0.000000,0.883170
80,0.033900,0.030588,0.439179,0.878358,0.878358,nan,0.878358,0.000000,0.878358
100,0.024800,0.029996,0.438913,0.877826,0.877826,nan,0.877826,0.000000,0.877826
120,0.037500,0.028461,0.427793,0.855586,0.855586,nan,0.855586,0.000000,0.855586
140,0.046300,0.028781,0.430205,0.860410,0.860410,nan,0.860410,0.000000,0.860410
160,0.023700,0.033380,0.411632,0.823265,0.823265,nan,0.823265,0.000000,0.823265
180,0.030900,0.027662,0.428238,0.856477,0.856477,nan,0.856477,0.000000,0.856477
200,0.023700,0.029923,0.424521,0.849043,0.849043,nan,0.849043,0.000000,0.849043


/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and ret

TrainOutput(global_step=400, training_loss=0.03607273187022656, metrics={'train_runtime': 2278.3185, 'train_samples_per_second': 0.702, 'train_steps_per_second': 0.176, 'total_flos': 2.80447288344576e+16, 'train_loss': 0.03607273187022656, 'epoch': 50.0})

In [13]:
# Push the fine-tuned model to Hugging Face
kwargs = {
    "tags": ["vision", "image-segmentation"],
    "finetuned_from": pretrained_model_name,
    "dataset": hf_dataset_identifier,
}

processor.push_to_hub(hub_model_id)
trainer.push_to_hub(**kwargs)

training_args.bin:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

events.out.tfevents.1707141837.0a092a314f00.35.0:   0%|          | 0.00/81.3k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/jacquelinegrimm/segformer-b0-finetuned-arabidopsis-roots-v03/commit/41e902f171e8da3858044acc5f254630ea41e67b', commit_message='End of training', commit_description='', oid='41e902f171e8da3858044acc5f254630ea41e67b', pr_url=None, pr_revision=None, pr_num=None)